In [1]:
#!/usr/bin/env python3

import ctypes
import os
from IPython.display import HTML, display
def bigprint(text, size=24):
    display(HTML(f"<p style='font-size:{size}px;'>{text}</p>"))
# --- Configuration for sphincs-shake-128s-simple ---
# These MUST match the parameters the library was compiled with
CRYPTO_PUBLICKEYBYTES = 32
CRYPTO_SECRETKEYBYTES = 64
CRYPTO_BYTES = 7856 # Signature length
# Path to the shared library relative to this script
LIB_PATH = os.path.join("ref", "libpqsphincs.so")
# --- End Configuration ---
print(f"--- Loading shared library: {LIB_PATH} ---")
try:
    sphincs_lib = ctypes.CDLL(LIB_PATH)
except OSError as e:
    print(f"Error loading library: {e}")
    print("Did you compile it using 'make -C ref lib PARAMS=sphincs-shake-128s THASH=simple'?")

print("--- Defining function prototypes (ctypes) ---")

# int crypto_sign_keypair(unsigned char *pk, unsigned char *sk);
sphincs_lib.crypto_sign_keypair.argtypes = [
    ctypes.POINTER(ctypes.c_ubyte), # pk
    ctypes.POINTER(ctypes.c_ubyte)  # sk
]
sphincs_lib.crypto_sign_keypair.restype = ctypes.c_int

# int crypto_sign(unsigned char *sm, unsigned long long *smlen,
#                 const unsigned char *m, unsigned long long mlen,
#                 const unsigned char *sk);
sphincs_lib.crypto_sign.argtypes = [
    ctypes.POINTER(ctypes.c_ubyte), # sm (signed message output)
    ctypes.POINTER(ctypes.c_ulonglong), # smlen (signed message length output)
    ctypes.POINTER(ctypes.c_ubyte), # m (message input)
    ctypes.c_ulonglong,             # mlen (message length input)
    ctypes.POINTER(ctypes.c_ubyte)  # sk (secret key input)
]
sphincs_lib.crypto_sign.restype = ctypes.c_int

# int crypto_sign_open(unsigned char *m, unsigned long long *mlen,
#                      const unsigned char *sm, unsigned long long smlen,
#                      const unsigned char *pk);
sphincs_lib.crypto_sign_open.argtypes = [
    ctypes.POINTER(ctypes.c_ubyte), # m (message output)
    ctypes.POINTER(ctypes.c_ulonglong), # mlen (message length output)
    ctypes.POINTER(ctypes.c_ubyte), # sm (signed message input)
    ctypes.c_ulonglong,             # smlen (signed message length input)
    ctypes.POINTER(ctypes.c_ubyte)  # pk (public key input)
]
sphincs_lib.crypto_sign_open.restype = ctypes.c_int

# --- Allocate Buffers ---
print("--- Allocating memory buffers ---")
pk_buf = (ctypes.c_ubyte * CRYPTO_PUBLICKEYBYTES)()
sk_buf = (ctypes.c_ubyte * CRYPTO_SECRETKEYBYTES)()

original_message = b"Hello, I am at University of Rochester."
mlen = len(original_message)
# Buffer for the original message (needs to be mutable for ctypes)
# Create buffer as c_ubyte array directly
m_buf = (ctypes.c_ubyte * mlen)(*original_message)

# Buffer for the signed message (signature + original message)
sm_buf = (ctypes.c_ubyte * (CRYPTO_BYTES + mlen))()
smlen = ctypes.c_ulonglong() # To receive the signed message length

# Buffer to receive the opened message after verification
mout_buf = (ctypes.c_ubyte * mlen)() # Should be same size as original
mlen_out = ctypes.c_ulonglong() # To receive the opened message length

--- Loading shared library: ref/libpqsphincs.so ---
--- Defining function prototypes (ctypes) ---
--- Allocating memory buffers ---


In [2]:
# 1. Key Generation
bigprint("\n--- (1) Generating Keys ---")
ret = sphincs_lib.crypto_sign_keypair(pk_buf, sk_buf)
if ret == 0:
    bigprint("Key generation successful.")
    pk_hex = bytes(pk_buf).hex()
    sk_hex = bytes(sk_buf).hex()
    bigprint(f"Public Key ({CRYPTO_PUBLICKEYBYTES} bytes): {pk_hex[:]}")
    bigprint(f"Secret Key ({CRYPTO_SECRETKEYBYTES} bytes): {sk_hex[:]}")
else:
    bigprint(f"Key generation failed! Return code: {ret}")
    
# check if length of pk and sk are correct
if len(pk_hex) != CRYPTO_PUBLICKEYBYTES * 2:
    bigprint(f"Public Key length mismatch! Expected {CRYPTO_PUBLICKEYBYTES}, Got {len(pk_hex) //2}")
if len(sk_hex) != CRYPTO_SECRETKEYBYTES * 2:
    bigprint(f"Secret Key length mismatch! Expected {CRYPTO_SECRETKEYBYTES}, Got {len(sk_hex) //2}")


In [3]:
bigprint("\n--- (2) Signing Original Message ---")
bigprint(f"Original message: {original_message.decode()}")
ret = sphincs_lib.crypto_sign(sm_buf, ctypes.byref(smlen), m_buf, mlen, sk_buf)
if ret == 0:
    bigprint("Signing successful.")
    bigprint(f"Signed message length: {smlen.value}")
    # Extract signature (first CRYPTO_BYTES)
    signature = bytes(sm_buf[:CRYPTO_BYTES])
    sig_hex = signature.hex()
    bigprint(f"Signature ({CRYPTO_BYTES} bytes): {sig_hex[:]}...{sig_hex[:]}")
    # Verify signed message length (should be sig bytes + msg bytes)
    if smlen.value != CRYPTO_BYTES + mlen:
            bigprint(f"Warning: Signed message length mismatch! Expected {CRYPTO_BYTES + mlen}, Got {smlen.value}")
else:
    bigprint(f"Signing failed! Return code: {ret}")

In [4]:
# 3. Verify Original Message (Should be Correct)
bigprint("\n--- (3) Verifying Original Message ---")
ret = sphincs_lib.crypto_sign_open(mout_buf, ctypes.byref(mlen_out), sm_buf, smlen.value, pk_buf)
if ret == 0:
    bigprint("Verification successful (Return code 0).")
    opened_message = bytes(mout_buf[:mlen_out.value])
    bigprint(f"Opened message length: {mlen_out.value}")
    bigprint(f"Opened message: {opened_message.decode()}")
    if mlen_out.value == mlen and opened_message == original_message:
        bigprint("Verification Result: CORRECT (Message matches)")
    else:
        bigprint("Verification Result: FAILED (Message mismatch or length error despite return code 0!)")
else:
    bigprint(f"Verification failed! Return code: {ret}")
    bigprint("Verification Result: CORRECT (Verification failed as expected for a valid signature? This is odd.)") # Should not happen for valid sig


In [ ]:
# 3. Verify Original Message (Should be Correct)
bigprint("\n--- (4) Verifying Modified Signature-Message Pair ---")
# flip a bit in the sm buf and verify the message
sm_buf[0] ^= 1
ret = sphincs_lib.crypto_sign_open(mout_buf, ctypes.byref(mlen_out), sm_buf, smlen.value, pk_buf)
if ret == 0:
    print("Verification successful (Return code 0).")
    opened_message = bytes(mout_buf[:mlen_out.value])
    print(f"Opened message length: {mlen_out.value}")
else:
    print(f"Verification failed! Return code: {ret}")